In [1]:
'''
A Recurrent Neural Network (LSTM) implementation example using TensorFlow..
Next word prediction after n_input words learned from text file.
A story is automatically generated if the predicted word is fed back as input.
Author: Rowel Atienza
Project: https://github.com/roatienza/Deep-Learning-Experiments
'''

from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
import collections
import time

In [2]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"


# Target log path
logs_path = '../tmp'
writer = tf.summary.FileWriter(logs_path)

# Text file containing words for training
training_file = 'belling_the_cat.txt'

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

training_data = read_data(training_file)
print("Loaded training data...")

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

Loaded training data...


In [3]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)

In [28]:
dictionary

{',': 0,
 '.': 2,
 '?': 62,
 'a': 6,
 'about': 68,
 'ago': 82,
 'agree': 95,
 'all': 17,
 'always': 52,
 'an': 85,
 'and': 3,
 'another': 72,
 'applause': 46,
 'approach': 61,
 'approaches': 66,
 'at': 34,
 'attached': 89,
 'be': 63,
 'bell': 18,
 'but': 30,
 'by': 25,
 'case': 87,
 'cat': 13,
 'chief': 111,
 'common': 54,
 'consider': 37,
 'consists': 39,
 'could': 7,
 'council': 98,
 'danger': 71,
 'easily': 36,
 'easy': 44,
 'enemy': 21,
 'escape': 57,
 'from': 74,
 'general': 33,
 'got': 23,
 'had': 19,
 'he': 11,
 'her': 26,
 'i': 108,
 'if': 99,
 'impossible': 73,
 'in': 15,
 'is': 14,
 'it': 97,
 'know': 49,
 'last': 69,
 'long': 77,
 'looked': 100,
 'make': 101,
 'manner': 96,
 'means': 47,
 'measures': 106,
 'meet': 94,
 'met': 65,
 'mice': 31,
 'mouse': 16,
 'neck': 107,
 'neighbourhood': 67,
 'nobody': 84,
 'now': 51,
 'of': 27,
 'old': 22,
 'one': 40,
 'our': 58,
 'outwit': 43,
 'procured': 103,
 'proposal': 28,
 'propose': 20,
 'receive': 70,
 'remedies': 81,
 'retire': 38

In [4]:
# Parameters
learning_rate = 0.001
training_iters = 10000
display_step = 1000
n_input = 3

# number of units in RNN cell
n_hidden = 512

# tf Graph input
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):
    
    # reshape to [1, n_input]
    x = tf.reshape(x, [-1, n_input])
    print(x)

    # Generate a n_input-element sequence of inputs
    # (eg. [had] [a] [general] -> [20] [6] [33])
    x = tf.split(x,n_input,1)
    print(x)

    # 2-layer LSTM, each layer has n_hidden units.
    # Average Accuracy= 95.20% at 50k iter
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    print(rnn_cell)
    
    # 1-layer LSTM with n_hidden units but with lower accuracy.
    # Average Accuracy= 90.60% 50k iter
    # Uncomment line below to test but comment out the 2-layer rnn.MultiRNNCell above
    # rnn_cell = rnn.BasicLSTMCell(n_hidden)

    # generate prediction
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    print(outputs)

    # there are n_input outputs but
    # we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [5]:
pred = RNN(x, weights, biases)

Tensor("Reshape:0", shape=(?, 3), dtype=float32)
[<tf.Tensor 'split:0' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:1' shape=(?, 1) dtype=float32>, <tf.Tensor 'split:2' shape=(?, 1) dtype=float32>]
[<tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_2:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_5:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'rnn/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/Mul_8:0' shape=(?, 512) dtype=float32>]


In [6]:
# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [16]:
n_input = 5
offset = random.randint(0,n_input+1)
symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]

In [27]:
dictionary

{',': 0,
 '.': 2,
 '?': 62,
 'a': 6,
 'about': 68,
 'ago': 82,
 'agree': 95,
 'all': 17,
 'always': 52,
 'an': 85,
 'and': 3,
 'another': 72,
 'applause': 46,
 'approach': 61,
 'approaches': 66,
 'at': 34,
 'attached': 89,
 'be': 63,
 'bell': 18,
 'but': 30,
 'by': 25,
 'case': 87,
 'cat': 13,
 'chief': 111,
 'common': 54,
 'consider': 37,
 'consists': 39,
 'could': 7,
 'council': 98,
 'danger': 71,
 'easily': 36,
 'easy': 44,
 'enemy': 21,
 'escape': 57,
 'from': 74,
 'general': 33,
 'got': 23,
 'had': 19,
 'he': 11,
 'her': 26,
 'i': 108,
 'if': 99,
 'impossible': 73,
 'in': 15,
 'is': 14,
 'it': 97,
 'know': 49,
 'last': 69,
 'long': 77,
 'looked': 100,
 'make': 101,
 'manner': 96,
 'means': 47,
 'measures': 106,
 'meet': 94,
 'met': 65,
 'mice': 31,
 'mouse': 16,
 'neck': 107,
 'neighbourhood': 67,
 'nobody': 84,
 'now': 51,
 'of': 27,
 'old': 22,
 'one': 40,
 'our': 58,
 'outwit': 43,
 'procured': 103,
 'proposal': 28,
 'propose': 20,
 'receive': 70,
 'remedies': 81,
 'retire': 38

In [18]:
symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

In [19]:
symbols_in_keys

array([[[ 6],
        [33],
        [98],
        [ 4],
        [37]]])

In [26]:
dictionary[ str(training_data[offset+n_input])]

60

In [21]:
symbols_out_onehot = np.zeros([vocab_size], dtype=float)
symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

In [22]:
symbols_out_onehot

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0

    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_input+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")